In [1]:
import pandas as pd
import syft as sy
import numpy as np
sy.logger.remove()

Seeding DB with roles!


### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into italy... done!


In [3]:
ca.datasets

,name,id,tags,data,description
0,Canada Trade Data - First few rows,dcdb740b-b77e-4011-ae93-01125d0de1be,[],"[{'name': 'feb2020-10', 'id': '81d7a910-f868-4...",A collection of reports from Canada's statisti...
1,Canada Trade Data - First few rows,d3b56e0b-f7bc-4224-929b-c8fe28082b4a,[],"[{'name': 'feb2020-10', 'id': '85269e80-92bd-4...",A collection of reports from Canada's statisti...


In [4]:
it.datasets

,name,data,description,id,tags
0,Italy Trade Data - First few rows,"[{'name': 'feb2020-10', 'id': '284747ee-5c99-4...",A collection of reports from iStat's statistic...,518f50b2-4a3d-44f7-94f1-5d18a7189e30,[]
1,Italy Trade Data - First few rows,"[{'name': 'feb2020-10', 'id': '730a08b2-b16c-4...",A collection of reports from iStat's statistic...,71fb3df8-d6cc-4f1d-90bb-9b03b3261b57,[]


In [5]:
ca.privacy_budget

41.592815128439895

In [6]:
it.privacy_budget

106.96564873038481

In [13]:
ca_data = ca.datasets[-1]['feb2020-10']
it_data = it.datasets[-1]['feb2020-10']

In [14]:
# this should be impossible - we're adding data that's located on two different computers
result = ca_data + it_data

In [15]:
public_result = result.publish(sigma=2)

In [16]:
ca.privacy_budget

41.592815128439895

In [17]:
it.privacy_budget

41.592815128439895

In [18]:
public_result.get()

array([ 0,  1,  1,  2,  2, -3,  2,  0,  1, -1], dtype=int32)